<a href="https://colab.research.google.com/github/ishandahal/stats453-deep_learning_torch/blob/main/Conv/All_Convolutional__Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## All Convolutional Neural Network

## Imports

In [1]:
import time
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings and Dataset

In [4]:
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 15
batch_size = 256

# Architecture
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


In [5]:
## Model 

class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.num_classes = num_classes
        # calculate same padding: 
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2

        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1) 
        
        # 28x28x4 => 14x14x4
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=4,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1)
        # 14x14x4 => 14x14x8 
        self.conv_3 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1)
        # 14x14x8 => 7x7x8
        self.conv_4 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=8,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1)
        # 7x7x8 => 7x7x16
        self.conv_5 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1)
        # 7x7x16 => 4x4x16
        self.conv_6 = torch.nn.Conv2d(in_channels=16,
                                      out_channels=16,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1)
        # 4x4x16 => 4x4xnum_classes
        self.conv_7 = torch.nn.Conv2d(in_channels=16,
                                      out_channels=self.num_classes,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1)
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)

        out = self.conv_2(out)
        out = F.relu(out)

        out = self.conv_3(out)
        out = F.relu(out)

        out = self.conv_4(out)
        out = F.relu(out)

        out = self.conv_5(out)
        out = F.relu(out)

        out = self.conv_6(out)
        out = F.relu(out)

        out = self.conv_7(out)
        out = F.relu(out)

        logits = F.adaptive_avg_pool2d(out, 1)
        # drop width
        logits.squeeze_(-1)
        # drop height
        logits.squeeze_(-1)
        probas = torch.softmax(logits, dim=1)

        return logits, probas

torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [6]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    
    
start_time = time.time()
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))
    
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/015 | Batch 000/235 | Cost: 2.3058
Epoch: 001/015 | Batch 050/235 | Cost: 2.2890
Epoch: 001/015 | Batch 100/235 | Cost: 2.0323
Epoch: 001/015 | Batch 150/235 | Cost: 1.7682
Epoch: 001/015 | Batch 200/235 | Cost: 1.4839
Epoch: 001/015 training accuracy: 59.61%
Time elapsed: 0.17 min
Epoch: 002/015 | Batch 000/235 | Cost: 1.3028
Epoch: 002/015 | Batch 050/235 | Cost: 1.2055
Epoch: 002/015 | Batch 100/235 | Cost: 1.0452
Epoch: 002/015 | Batch 150/235 | Cost: 0.9073
Epoch: 002/015 | Batch 200/235 | Cost: 0.8358
Epoch: 002/015 training accuracy: 72.81%
Time elapsed: 0.33 min
Epoch: 003/015 | Batch 000/235 | Cost: 0.7768
Epoch: 003/015 | Batch 050/235 | Cost: 0.8298
Epoch: 003/015 | Batch 100/235 | Cost: 0.8986
Epoch: 003/015 | Batch 150/235 | Cost: 0.9244
Epoch: 003/015 | Batch 200/235 | Cost: 0.8133
Epoch: 003/015 training accuracy: 79.94%
Time elapsed: 0.50 min
Epoch: 004/015 | Batch 000/235 | Cost: 0.5969
Epoch: 004/015 | Batch 050/235 | Cost: 0.5198
Epoch: 004/015 | Batch 100

# Evaluation

In [7]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 87.17%
